# axerunt

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import reduce
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('axerunt_test.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
from dask import delayed

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, is_valid, is_type, null_fn, identity_fn, pd_abs, pd_is_empty, ser_range_center_clip
from common_util import str_now_dtz, last_commit_dtz, load_json, dump_json, pd_slot_shift, arr_nonzero, get_variants, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import GRAPHS_DIR, HISTORY_DIR, TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import get_rm_keychain, process_transform
from runt_util import RUNT_TYPE_MAPPING, RUNTFormatError, get_ser_fn

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.0
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'impo

CRITICAL:root:script location: /home/kev/crunch/mutate/axerunt_test.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Init DataAPI

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
graphs, transforms = get_graphs(), get_transforms()

## #Set Transform

In [3]:
info = transforms['dohlca']

In [4]:
info

{'meta': {'rec_fmt': 'dohlca',
  'var_fmt': 'list',
  'res_freq': 'D',
  'info': 'daily open, high, low, close, and average aggregation'},
 'fn': {'ser_fn': ['sr', 'sr', 'sr', 'sr', 'stat'],
  'df_fn': 'guax',
  'col_fn': 'ohlca',
  'freq': 'D'},
 'axe': {'rm': ['hrm', 'hrm'],
  'src': ['root', 'root_split_ohlca'],
  'dst': ['dohlca', 'dohlca']},
 'var': [{'val': [True], 'flt': [0]},
  {'val': [True], 'flt': ['h']},
  {'val': [True], 'flt': ['l']},
  {'val': [True], 'flt': [-1]},
  {'stat_type': ['avg']}]}

# System Test Transform

In [5]:
res_rcs, res_dfs = list(process_transform(info, dump=False))

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


INFO:root:data: ['dow_jones', 'root', 'root_split_ohlca', 'join', 'pba']
DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'pba', 'pba']]
DEBUG:root:pre-runt:                            pba_open  pba_high   pba_low  pba_close  \
id                                                                   
1998-01-02 14:00:00+00:00   7910.24   7952.59   7910.24    7923.19   
1998-01-02 15:00:00+00:00   7923.19   7923.19   7880.85    7917.22   
1998-01-02 16:00:00+00:00   7917.22   7931.66   7914.48    7916.22   
1998-01-02 17:00:00+00:00   7915.72   7940.38   7915.72    7934.90   
1998-01-02 18:00:00+00:00   7934.65   7935.40   7916.22    7916.22   
1998-01-02 19:00:00+00:00   7916.22   7920.20   7896.29    7908.99   
1998-01-02 20:00:00+00:00   7910.24   7957.07   7910.24    7955.33   
1998-01-02 21:00:00+00:00   7956.08   7965.04   7955.33    7965.04   
1998-01-05 14:00:00+00:00   7967.04   7989.96   7952.59    7963.80   
1998-01-05 15:00:00+00:00   7965.04   8010.88   7965.04    7997.43   
1

DEBUG:root:variant: ({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})
DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'stat_type': 'avg'}


/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:407: FutureWarning: The default of the 'keep_tz' keyword will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()
/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:415: FutureWarning: The default of the 'keep_tz' keyword will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  dindex = axis_index.to_series()


DEBUG:root:mutate_type: dohlca
DEBUG:root:post-runt:                            pba_open  pba_high   pba_low  pba_close  \
id                                                                   
2009-01-02 00:00:00+00:00   8771.77   9065.28   8761.10    9034.69   
2009-01-05 00:00:00+00:00   9027.13   9033.66   8892.68    8952.89   
2009-01-06 00:00:00+00:00   8954.57   9087.82   8941.11    9015.10   
2009-01-07 00:00:00+00:00   8996.94   8996.94   8720.00    8769.70   
2009-01-08 00:00:00+00:00   8769.94   8769.94   8651.27    8742.46   
2009-01-09 00:00:00+00:00   8738.80   8769.07   8577.27    8599.18   
2009-01-12 00:00:00+00:00   8599.26   8600.37   8421.24    8473.97   
2009-01-13 00:00:00+00:00   8474.61   8521.44   8377.20    8448.56   
2009-01-14 00:00:00+00:00   8446.01   8446.01   8140.88    8200.14   
2009-01-15 00:00:00+00:00   8196.24   8285.92   7995.29    8212.49   
2009-01-16 00:00:00+00:00   8215.67   8340.48   8109.58    8281.22   
2009-01-20 00:00:00+00:00   8279.63  

...

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'trmi3_mkt3_social_bondBuzz']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - data_df.index: DatetimeIndex(['2003-10-26 17:00:00+00:00', '2017-12-16 16:00:00+00:00',
               '2017-12-16 17:00:00+00:00', '2017-12-16 18:00:00+00:00',
               '2017-12-16 19:00:00+00:00', '2017-12-16 20:00:00+00:00',
               '2017-12-16 21:00:00+00:00', '2017-12-16 22:00:00+00:00',
               '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                            mkt3_bondBuzz_S
id                                        
1998-01-02 14:00:00+00:00              0.0
1998-01-02 15:00:00+00:00              0.0
1998-01-02 16:00:00+00:00              7.0
1998-01-02 17:00:00+00:00              7.0
1998-01-02 18:00:00+00:00              7.0
1998-01-02 19:00:00+00:00              7.0
1998-01-02 20:00:00+00:00         

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'trmi3_mkt3_social_ratesBuzz']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - data_df.index: DatetimeIndex(['2003-10-26 17:00:00+00:00', '2017-12-16 16:00:00+00:00',
               '2017-12-16 17:00:00+00:00', '2017-12-16 18:00:00+00:00',
               '2017-12-16 19:00:00+00:00', '2017-12-16 20:00:00+00:00',
               '2017-12-16 21:00:00+00:00', '2017-12-16 22:00:00+00:00',
               '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                            mkt3_ratesBuzz_S
id                                         
1998-01-02 14:00:00+00:00               4.0
1998-01-02 15:00:00+00:00               4.0
1998-01-02 16:00:00+00:00               4.0
1998-01-02 17:00:00+00:00               4.0
1998-01-02 18:00:00+00:00               1.0
1998-01-02 19:00:00+00:00               3.0
1998-01-02 20:00:00+00:00

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'trmi3_mkt3_social_stockIndexBuzz']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - data_df.index: DatetimeIndex(['2003-10-26 17:00:00+00:00', '2017-12-16 16:00:00+00:00',
               '2017-12-16 17:00:00+00:00', '2017-12-16 18:00:00+00:00',
               '2017-12-16 19:00:00+00:00', '2017-12-16 20:00:00+00:00',
               '2017-12-16 21:00:00+00:00', '2017-12-16 22:00:00+00:00',
               '2017-12-17 14:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='id', freq=None)
DEBUG:root:pre-runt:                            mkt3_stockIndexBuzz_S
id                                              
1998-01-02 14:00:00+00:00                   19.0
1998-01-02 15:00:00+00:00                   30.0
1998-01-02 16:00:00+00:00                   31.0
1998-01-02 17:00:00+00:00                   48.0
1998-01-02 18:00:00+00:00                   48.0
1998-01-02 19:00:00+00:00    

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'vol']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'vol', 'vol']]
DEBUG:root:pre-runt:                            vol_open  vol_high  vol_low  vol_close  \
id                                                                  
1998-01-02 15:00:00+00:00     25.80     26.47    25.28      25.48   
1998-01-02 16:00:00+00:00     25.33     25.66    25.10      25.63   
1998-01-02 17:00:00+00:00     25.63     25.66    25.04      25.15   
1998-01-02 18:00:00+00:00     25.18     25.49    24.85      25.30   
1998-01-02 19:00:00+00:00     25.33     25.65    25.11      25.32   
1998-01-02 20:00:00+00:00     25.28     25.28    24.06      24.14   
1998-01-05 14:00:00+00:00     24.84     24.84    24.62      24.62   
1998-01-05 16:00:00+00:00     23.81     23.81    23.40      23.43   
1998-01-05 17:00:00+00:00     23.45     24.19    23.29      24.09   
1998-01-05 18:00:00+00:00     24.17     24.47    23.80      24.44   
1998-01-05 19:00:00

DEBUG:root:variant: ({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})
DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'stat_type': 'avg'}
DEBUG:root:mutate_type: dohlca
DEBUG:root:post-runt:                            vol_open  vol_high  vol_low  vol_close  \
id                                                                  
2009-01-02 00:00:00+00:00     39.58     39.82    36.88      39.19   
2009-01-05 00:00:00+00:00     39.24     40.22    38.30      39.08   
2009-01-06 00:00:00+00:00     38.06     39.33    37.34      38.56   
2009-01-07 00:00:00+00:00     38.56     43.82    38.56      43.39   
2009-01-08 00:00:00+00:00     43.38     44.60    42.56      42.56   
2009-01-09 00:00:00+00:00     41.18     43.13    41.05      42.82   
2009-01-12 00:00:00+00:00     

In [8]:
ckey = list(res_dfs.keys())[1]
display(ckey)
display(res_dfs[ckey])

['dow_jones',
 'root',
 'root_split_ohlca',
 'join',
 'trmi3_etf3_news_buzz',
 "({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})"]

,etf3_buzz_N_open,etf3_buzz_N_high,etf3_buzz_N_low,etf3_buzz_N_close,etf3_buzz_N_avgPrice
id,,,,,
2009-01-01 00:00:00+00:00,38915.800000,38915.800000,25883.900000,25883.900000,33894.466667
2009-01-02 00:00:00+00:00,18739.900000,27513.900000,18739.900000,27513.900000,22591.411111
2009-01-03 00:00:00+00:00,29957.900000,29957.900000,18231.300000,18231.300000,24735.444444
2009-01-04 00:00:00+00:00,8826.700000,8974.900000,7486.800000,7486.800000,8438.822222
2009-01-05 00:00:00+00:00,16674.300000,55199.000000,16674.300000,55199.000000,34845.988889
2009-01-06 00:00:00+00:00,81419.300000,81976.000000,73283.800000,74108.100000,76423.611111
2009-01-07 00:00:00+00:00,75924.100000,84207.800000,75924.100000,82753.700000,80456.277778
2009-01-08 00:00:00+00:00,90980.800000,98514.400000,90980.800000,98514.400000,94778.622222
2009-01-09 00:00:00+00:00,95265.500000,95265.500000,76354.900000,76354.900000,85722.222222


In [16]:
d[list(d.keys())[0]].dropna()

,pba_lh
id,
2009-01-02 14:00:00+00:00,0.006455
2009-01-02 15:00:00+00:00,0.016764
2009-01-02 16:00:00+00:00,0.019809
2009-01-02 17:00:00+00:00,0.021737
2009-01-02 18:00:00+00:00,0.025165
2009-01-02 19:00:00+00:00,0.028637
2009-01-02 20:00:00+00:00,0.034719
2009-01-02 21:00:00+00:00,0.031228
2009-01-05 14:00:00+00:00,0.012724


# Debug Transform - Init Transform Settings

In [5]:
meta, fn, axe, var = info['meta'], info['fn'], info['axe'], info['var']

if (meta['var_fmt']=='list' and any([s in meta['rec_fmt'] for s in ('{', '}')])):
    error_msg = 'cannot mix var_fmt==\'list\' with a parameter-inputed rec_fmt'
    logging.error(error_msg)
    raise RUNTFormatError(error_msg)

### Loading transform, apply, and frequency settings

In [6]:
rtype_fn = RUNT_TYPE_MAPPING.get(fn['df_fn'])
ser_fn_str, col_fn_str = fn['ser_fn'], fn['col_fn']
freq, res_freq = fn['freq'], meta['res_freq']

### Variants and type format string

In [7]:
variants = get_variants(var, meta['var_fmt'])
print(variants)
type_str = meta['rec_fmt']

[{'rank_type': 'mx', 'num_periods': 5}, {'rank_type': 'mx', 'num_periods': 20}, {'rank_type': 'mx', 'num_periods': 240}, {'rank_type': 'mx', 'num_periods': -1}]


### Prep input data and row masks

In [8]:
srcs = axe['src'] if (is_type(axe['src'][0], list)) else [axe['src']]
rm_src = axe['rm']
if (is_valid(rm_src)):
    rm_rcs, rm_dfs = DataAPI.axe_load(rm_src, lazy=False)

## #Debug Transform - Load Input Data

In [9]:
src = srcs[0]
src_rcs, src_dfs = DataAPI.axe_load(src, lazy=False)
dfkeys = list(src_dfs.keys())

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [10]:
keychain = dfkeys[0]
src_df = src_dfs[keychain]

### Row Mask Input Data

In [11]:
if (is_valid(rm_src)):
    rm_keychain = get_rm_keychain(keychain, rm_dfs.keys())
    rm_df = rm_dfs[rm_keychain].dropna()
    rm_src_diff = rm_df.index.difference(src_df.index)
    if (len(rm_src_diff)>0):
        logging.debug('rm_df.index - src_df.index: {}'.format(str(rm_src_diff)))
        src_df = src_df.loc[src_df.index & rm_df.index, :].dropna(axis=0, how='all')
    else:
        src_df = src_df.loc[rm_df.index, :].dropna(axis=0, how='all')

In [12]:
src_df.head(10)

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-01-02 00:00:00+00:00,8771.77,9065.28,8761.10,9034.69,8934.514025
2009-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2009-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2009-01-05 00:00:00+00:00,9027.13,9033.66,8892.68,8952.89,8964.985225
2009-01-06 00:00:00+00:00,8954.57,9087.82,8941.11,9015.10,9009.688837
2009-01-07 00:00:00+00:00,8996.94,8996.94,8720.00,8769.70,8820.237200
2009-01-08 00:00:00+00:00,8769.94,8769.94,8651.27,8742.46,8700.459338
2009-01-09 00:00:00+00:00,8738.80,8769.07,8577.27,8599.18,8640.200900
2009-01-10 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN


### #Set Variant

In [13]:
variant = variants[0]
print(variant)

{'rank_type': 'mx', 'num_periods': 5}


## Debug Transform - Load Functions and Variants

In [14]:
fns = get_ser_fn(fn['ser_fn'], variant)
print(fns)

DEBUG:root:subvar: {'rank_type': 'mx', 'num_periods': 5}
[<function window_rank.<locals>.fn at 0x7f8e1febb598>]


## Debug Transform - Isolated Transform Testing

In [91]:
logging.info(fn['df_fn'])
logging.info(freq)
if (fn['df_fn'] == 'rut'):
    for i, col in enumerate(src_df.columns):
        ser = src_df.iloc[:, i]
        #logging.info('{c}: {s}'.format(c=col, s=ser))
        #display(fns[0](ser))
        tser = pd.Series(range(100))
        rtser = pd.Series(reversed(range(100)))
        out = window_rank('od', -1)(ser)
        display(out)
elif (fn['df_fn'] == 'rbt'):
    pass
elif (fn['df_fn'] == 'gut'):
    pass
elif (fn['df_fn'] == 'gua'):
    for i, col in enumerate(src_df.columns):
        ser = src_df.iloc[:, i]
        logging.info('{c}: {s}'.format(c=col, s=ser))
        #print(ser.groupby(pd.Grouper(freq=freq)).agg(fns[-pd.Series(data=[], index=[]).idxmax()]))

INFO:root:rut
INFO:root:None


id
2009-01-02 00:00:00+00:00    1.000000
2009-01-05 00:00:00+00:00    1.000000
2009-01-06 00:00:00+00:00    0.666667
2009-01-07 00:00:00+00:00    0.750000
2009-01-08 00:00:00+00:00    0.200000
2009-01-09 00:00:00+00:00    0.166667
2009-01-12 00:00:00+00:00    0.142857
2009-01-13 00:00:00+00:00    0.125000
2009-01-14 00:00:00+00:00    0.111111
2009-01-15 00:00:00+00:00    0.100000
2009-01-16 00:00:00+00:00    0.181818
2009-01-20 00:00:00+00:00    0.250000
2009-01-21 00:00:00+00:00    0.076923
2009-01-22 00:00:00+00:00    0.285714
2009-01-23 00:00:00+00:00    0.133333
2009-01-26 00:00:00+00:00    0.125000
2009-01-27 00:00:00+00:00    0.235294
2009-01-28 00:00:00+00:00    0.277778
2009-01-29 00:00:00+00:00    0.526316
2009-01-30 00:00:00+00:00    0.250000
2009-02-02 00:00:00+00:00    0.095238
2009-02-03 00:00:00+00:00    0.045455
2009-02-04 00:00:00+00:00    0.173913
2009-02-05 00:00:00+00:00    0.125000
2009-02-06 00:00:00+00:00    0.200000
2009-02-09 00:00:00+00:00    0.615385
2009-02-1

id
2009-01-02 00:00:00+00:00    1.000000
2009-01-05 00:00:00+00:00    0.500000
2009-01-06 00:00:00+00:00    1.000000
2009-01-07 00:00:00+00:00    0.250000
2009-01-08 00:00:00+00:00    0.200000
2009-01-09 00:00:00+00:00    0.166667
2009-01-12 00:00:00+00:00    0.142857
2009-01-13 00:00:00+00:00    0.125000
2009-01-14 00:00:00+00:00    0.111111
2009-01-15 00:00:00+00:00    0.100000
2009-01-16 00:00:00+00:00    0.181818
2009-01-20 00:00:00+00:00    0.166667
2009-01-21 00:00:00+00:00    0.076923
2009-01-22 00:00:00+00:00    0.071429
2009-01-23 00:00:00+00:00    0.066667
2009-01-26 00:00:00+00:00    0.187500
2009-01-27 00:00:00+00:00    0.176471
2009-01-28 00:00:00+00:00    0.500000
2009-01-29 00:00:00+00:00    0.473684
2009-01-30 00:00:00+00:00    0.100000
2009-02-02 00:00:00+00:00    0.047619
2009-02-03 00:00:00+00:00    0.090909
2009-02-04 00:00:00+00:00    0.173913
2009-02-05 00:00:00+00:00    0.083333
2009-02-06 00:00:00+00:00    0.520000
2009-02-09 00:00:00+00:00    0.538462
2009-02-1

id
2009-01-02 00:00:00+00:00    1.000000
2009-01-05 00:00:00+00:00    1.000000
2009-01-06 00:00:00+00:00    1.000000
2009-01-07 00:00:00+00:00    0.250000
2009-01-08 00:00:00+00:00    0.200000
2009-01-09 00:00:00+00:00    0.166667
2009-01-12 00:00:00+00:00    0.142857
2009-01-13 00:00:00+00:00    0.125000
2009-01-14 00:00:00+00:00    0.111111
2009-01-15 00:00:00+00:00    0.100000
2009-01-16 00:00:00+00:00    0.181818
2009-01-20 00:00:00+00:00    0.083333
2009-01-21 00:00:00+00:00    0.076923
2009-01-22 00:00:00+00:00    0.214286
2009-01-23 00:00:00+00:00    0.066667
2009-01-26 00:00:00+00:00    0.375000
2009-01-27 00:00:00+00:00    0.411765
2009-01-28 00:00:00+00:00    0.555556
2009-01-29 00:00:00+00:00    0.473684
2009-01-30 00:00:00+00:00    0.250000
2009-02-02 00:00:00+00:00    0.047619
2009-02-03 00:00:00+00:00    0.090909
2009-02-04 00:00:00+00:00    0.173913
2009-02-05 00:00:00+00:00    0.041667
2009-02-06 00:00:00+00:00    0.480000
2009-02-09 00:00:00+00:00    0.692308
2009-02-1

id
2009-01-02 00:00:00+00:00    1.000000
2009-01-05 00:00:00+00:00    0.500000
2009-01-06 00:00:00+00:00    0.666667
2009-01-07 00:00:00+00:00    0.250000
2009-01-08 00:00:00+00:00    0.200000
2009-01-09 00:00:00+00:00    0.166667
2009-01-12 00:00:00+00:00    0.142857
2009-01-13 00:00:00+00:00    0.125000
2009-01-14 00:00:00+00:00    0.111111
2009-01-15 00:00:00+00:00    0.200000
2009-01-16 00:00:00+00:00    0.272727
2009-01-20 00:00:00+00:00    0.083333
2009-01-21 00:00:00+00:00    0.307692
2009-01-22 00:00:00+00:00    0.142857
2009-01-23 00:00:00+00:00    0.133333
2009-01-26 00:00:00+00:00    0.187500
2009-01-27 00:00:00+00:00    0.294118
2009-01-28 00:00:00+00:00    0.555556
2009-01-29 00:00:00+00:00    0.263158
2009-01-30 00:00:00+00:00    0.100000
2009-02-02 00:00:00+00:00    0.047619
2009-02-03 00:00:00+00:00    0.227273
2009-02-04 00:00:00+00:00    0.130435
2009-02-05 00:00:00+00:00    0.208333
2009-02-06 00:00:00+00:00    0.600000
2009-02-09 00:00:00+00:00    0.576923
2009-02-1

id
2009-01-02 00:00:00+00:00    1.000000
2009-01-05 00:00:00+00:00    1.000000
2009-01-06 00:00:00+00:00    1.000000
2009-01-07 00:00:00+00:00    0.250000
2009-01-08 00:00:00+00:00    0.200000
2009-01-09 00:00:00+00:00    0.166667
2009-01-12 00:00:00+00:00    0.142857
2009-01-13 00:00:00+00:00    0.125000
2009-01-14 00:00:00+00:00    0.111111
2009-01-15 00:00:00+00:00    0.100000
2009-01-16 00:00:00+00:00    0.272727
2009-01-20 00:00:00+00:00    0.083333
2009-01-21 00:00:00+00:00    0.076923
2009-01-22 00:00:00+00:00    0.071429
2009-01-23 00:00:00+00:00    0.066667
2009-01-26 00:00:00+00:00    0.375000
2009-01-27 00:00:00+00:00    0.411765
2009-01-28 00:00:00+00:00    0.555556
2009-01-29 00:00:00+00:00    0.473684
2009-01-30 00:00:00+00:00    0.100000
2009-02-02 00:00:00+00:00    0.047619
2009-02-03 00:00:00+00:00    0.090909
2009-02-04 00:00:00+00:00    0.173913
2009-02-05 00:00:00+00:00    0.125000
2009-02-06 00:00:00+00:00    0.600000
2009-02-09 00:00:00+00:00    0.653846
2009-02-1